# DAT561 Final Project (Fall 2023)

**Note**: Please be creative in defining the new variables as part of the data manipulation and write your description at the end of each code as the comment. We will read your logic and description for the assessment.


**Part 1**: 
75 points (85 points with the extra credits in the Bonus Question)

In [2]:
import numpy as np
import pandas as pd

In [3]:
# Read the dataset here
Property_details = pd.read_csv("Property_details.csv")
# You may use Tkinter library
Order_details = pd.read_csv("Order_details.csv", encoding='ISO-8859-1')

### Question 1

**Part (a)**: How many properties are there in the region with the zip code of 84600? 

**Part (b)**: What is the mean, standard deviation, median, min, and max of “starratings” for all properties in the region with the zip code of 84600?


In [4]:
# Part (a):
properties_in_84600 = Property_details[Property_details["zipcode"] == 84600]
properties_in_84600.shape[0]

104

In [5]:
# Part (b):
properties_in_84600.groupby(by="zipcode")["starrating"].agg([np.mean, np.std, np.min, np.max])

,mean,std,amin,amax
zipcode,,,,
84600.0,3.461538,0.709479,2,4


### Question 2

**Part (a)**: Create a new column called "weekday", which is the day of the “reservation date” in one week (for example, if the reservation date is 2021/10/22, the corresponding value in the new column "weekday" should be “Fri”).

**Part (b)**: Which weekday receives an above-average number of reservations compared to all records?

In [6]:
# Part (a):
Order_details["reservation date"] = pd.to_datetime(Order_details["reservation date"])
Order_details["weekday"] = Order_details["reservation date"].dt.strftime('%a')
Order_details["weekday"]

0         Sat
1         Sat
2         Fri
3         Sat
4         Sat
         ... 
165868    Tue
165869    Tue
165870    Tue
165871    Tue
165872    Tue
Name: weekday, Length: 165873, dtype: object

In [7]:
# Part (b):
weekday_counts = Order_details["weekday"].value_counts()
average = weekday_counts.mean()
above_average = weekday_counts[weekday_counts > average]
above_average

Thu    26589
Mon    26539
Tue    24275
Wed    24082
Name: weekday, dtype: int64

### Question 3

**Part (a)**: In “roomamenities”, what are the top 10 common room amenities in single rooms (rooms with a maxoccupancy of 1)? A what about the 10 least common room amenities in single rooms? (For example, Air conditioning is one amenity)

**Part (b)**: What percentage does each type of room amenities occupy of the total number of reservations for single rooms?(Do not use the total number of amenities as denominator）

In [9]:
# You can deceide whether to display you output for 3(a) and 3(b) separately or together
Single_Room = Order_details[Order_details['maxoccupancy'] == 1]
amenities = Single_Room['roomamenities'].str.split(': ;').explode()
amenities = amenities[amenities.str.strip() != '']

top_10_common = amenities.value_counts().head(10)
least_10_common = amenities.value_counts().sort_values().head(10)

print("most common: ")
print(top_10_common)
print("")
print("least common: ")
print(least_10_common)

most common: 
Air conditioning            83913
In-room safe box            83913
Free Wi-Fi in all rooms!    82341
Hair dryer                  61079
Telephone                   57610
Toiletries                  57397
Desk                        55988
Heating                     55773
Closet                      52029
Satellite/cable channels    51822
Name: roomamenities, dtype: int64

least common: 
Open air bath privileges    12
Hot tub                     13
Vending machine             16
Humidifier                  21
TV [in bathroom]            24
Internet access ï¿?LAN      26
Scale                       35
Air purifier                46
First aid kit               56
In-room tablet              63
Name: roomamenities, dtype: int64


In [10]:
amenities_counts = amenities.value_counts()
total_amenities = amenities_counts.sum()
amenities_percentage = amenities_counts / total_amenities * 100
amenities_percentage

Air conditioning            6.455949
In-room safe box            6.455949
Free Wi-Fi in all rooms!    6.335005
Hair dryer                  4.699187
Telephone                   4.432295
                              ...   
TV [in bathroom]            0.001846
Humidifier                  0.001616
Vending machine             0.001231
Hot tub                     0.001000
Open air bath privileges    0.000923
Name: roomamenities, Length: 91, dtype: float64

### Question 4

**Part (a)**: For each property, there are some abnormal values of 0 in the “onsiteprice”. To better organize the data, you would like to create a new column “replaced onsiteprice” in the dataset by retaining the original non-zero “onsiteprice” of one specific property and replacing the zero value with its median of non-zero “onsiteprice”.

**Part (b)**: For each property, calculate the maximum and minimum value of “replaced onsiteprice”, and store these two into corresponding two columns named “Maximum” and “Minimum”. Then create a column named “Normalized Maximum” to store the normalized form of the “Maximum” column. You can use the formula below for the normalization (do not round the result). Store the “hotelcode”, “Maximum”, “Minimum”, “Normalized Maximum” to "Max_Min Price.csv". 
$ X_{norm} = \frac{X-X_{min}}{X_\max-X_{min}}$

In [12]:
# Part (a):
price_median_by_type = Order_details[Order_details['onsiteprice'] != 0].groupby('propertycode')['onsiteprice'].median()
Order_details['replaced_onsiteprice'] = Order_details['onsiteprice']

for property_code, median_price in price_median_by_type.items():
    condition = (Order_details['propertycode'] == property_code) & (Order_details['onsiteprice'] == 0)
    Order_details.loc[condition, 'replaced_onsiteprice'] = median_price

In [13]:
# Part (b):
max_min = Order_details.groupby('propertycode')['replaced_onsiteprice'].agg(['max', 'min'])
max_min.columns = ['Maximum', 'Minimum']

Order_details = Order_details.merge(max_min, on='propertycode', how='left')

# Normalized Maximum
Order_details['Normalized_Maximum'] = (Order_details['onsiteprice'] - Order_details['Minimum']) / (Order_details['Maximum'] - Order_details['Minimum'])

Order_details['hotelcode'] = Order_details['propertycode']
Order_details[['hotelcode', 'Maximum', 'Minimum', 'Normalized_Maximum']].to_csv('Max_Min_Price.csv', index=False)


### Question 5

**Part (a)**: A party of three is planning a trip. How many available hotels do offer a room with the “maxoccupancy” of 3 or more? Available hotel are those whose “propertype” are “Hotels”, “close” are “N”, and “hotelblock” are not “sold out” .

**Part (b)**: If this party does not want to pay a room for a “replaced onsiteprice” higher than 150 per night, how many hotels are still available? Use the maximum of “replaced onsiteprice” to compare with 150 due to price fluctuation. 


In [14]:
Order_x_Property = Order_details.merge(Property_details, how = "left", left_on = "propertycode", right_on = "propertyid")
Order_x_Property.head(3)

,id,propertycode,dtcollected,reservation date,los,guests,roomtype,onsiteprice,ratedescription,ratetype,...,propertyname,address,city,country,zipcode,propertytype,starrating,latitude,longitude,url
0,50677497,634876,10/12/2019 15:46,2019-11-02,1,1,Double Room,82.36,"Room size: 15 mÂ²/161 ftÂ², Shower, 1 king bed",Cancellation policy,...,The Old Cider House,25 Castle Street,Nether Stowey,United Kingdom,NaN,Hotels,4,51.150921,-3.15847,https://www.agoda.com/en-gb/the-old-cider-hous...
1,50672149,8328096,10/12/2019 15:47,2019-11-30,1,1,Vacation Home,636.09,"Shower, Kitchenette, 2 bedrooms, 1 double bed ...",Cancellation policy,...,Pantlleni,NaN,Beddgelert,United Kingdom,NaN,Hotels,3,53.042229,-4.03318,https://www.agoda.com/en-gb/pantlleni/hotel/be...
2,50643430,8323442,10/12/2019 15:47,2019-12-20,1,1,Vacation Home,591.74,"Shower, Kitchenette, 2 bedrooms, 1 double bed ...",Extra low price! (non-refundable),...,Willow Cottage,NaN,Beverley,United Kingdom,NaN,Hotels,3,53.843220,-0.49797,https://www.agoda.com/en-gb/willow-cottage/hot...


In [15]:
# Part (a):

available_hotels = Order_details[
    (Order_x_Property['propertytype'] == 'Hotels') &
    (Order_x_Property['closed'] == 'N') &
    (Order_x_Property['hotelblock'] != 'sold out') &
    (Order_x_Property['maxoccupancy'] >= 3)
]

len( available_hotels['propertycode'].unique() )

3085

In [16]:
# Part (b):

available_hotels_2 = Order_details[
    (Order_x_Property['propertytype'] == 'Hotels') &
    (Order_x_Property['closed'] == 'N') &
    (Order_x_Property['hotelblock'] != 'sold out') &
    (Order_x_Property['maxoccupancy'] >= 3) &
    (Order_x_Property['Maximum'] >= 150)
]

len( available_hotels_2['propertycode'].unique() )

1713

### Bonus Question:

Merge data, filter, groupby, merge three times

**Part (a)**: For each country, find the most expensive property by using “replaced onsiteprice”. Provide id, name, rating, city, country, zip code, address, and average “replaced onsiteprice” of these properties.

**Part (b)**: For each country, find the cheapest property by using “replaced onsiteprice”. Provide id, name, rating, city, country, zip code, address, and average “replaced onsiteprice” of these properties.

**Hint**: Each country has numbers of hotels, and each hotel has numbers of prices due to price fluctuation. You need to find the average “replaced onsiteprice” for each hotel first, and sort out the cheapest and the most expensive hotels then.



In [17]:
# Part (a):
each_country_each_hotel = Order_x_Property.groupby( by = ['country','propertyid'])['replaced_onsiteprice'].agg(np.mean).reset_index()
each_country_each_hotel

max_price_per_country = each_country_each_hotel.groupby(by = ['country'])['replaced_onsiteprice'].agg(np.max).reset_index()
max_price_per_country # no id now

# show id again
most_expensive_properties_id = max_price_per_country.merge(each_country_each_hotel, on=['country', 'replaced_onsiteprice'], how='left')
most_expensive_properties_id 

# whole info
most_expensive_properties = most_expensive_properties_id.merge(Order_x_Property, on=['country', 'replaced_onsiteprice', 'propertyid'], how='left')
most_expensive_properties

columns = ['propertyid', 'propertyname', 'starrating', 'city', 'country', 'zipcode', 'address', 'replaced_onsiteprice']
most_expensive_properties = most_expensive_properties[columns]
print(most_expensive_properties)


    propertyid       propertyname  starrating          city         country  \
0       130055                NaN         NaN           NaN         Austria   
1        49541                NaN         NaN           NaN         Belgium   
2      7503033                NaN         NaN           NaN        Bulgaria   
3       112561                NaN         NaN           NaN         Croatia   
4       152681                NaN         NaN           NaN          Cyprus   
5       367192                NaN         NaN           NaN  Czech Republic   
6      6720996                NaN         NaN           NaN         Denmark   
7         2202                NaN         NaN           NaN         Estonia   
8       365601  HavsVidden Resort         4.0          Geta         Finland   
9       442720       Hotel Astrid         4.0       Lourdes          France   
10      442720       Hotel Astrid         4.0       Lourdes          France   
11      442720       Hotel Astrid         4.0       

In [18]:
# Part (b):
each_country_each_hotel = Order_x_Property.groupby( by = ['country','propertyid'])['replaced_onsiteprice'].agg(np.mean).reset_index()
each_country_each_hotel

min_price_per_country = each_country_each_hotel.groupby(by = ['country'])['replaced_onsiteprice'].agg(np.min).reset_index()
min_price_per_country # no id now

# show id again
cheapest_properties_id = min_price_per_country.merge(each_country_each_hotel, on=['country', 'replaced_onsiteprice'], how='left')
cheapest_properties_id 

# whole info
cheapest_properties = cheapest_properties_id.merge(Order_x_Property, on=['country', 'replaced_onsiteprice', 'propertyid'], how='left')
cheapest_properties

columns = ['propertyid', 'propertyname', 'starrating', 'city', 'country', 'zipcode', 'address', 'replaced_onsiteprice']
cheapest_properties = cheapest_properties[columns]
print(cheapest_properties)

    propertyid                            propertyname  starrating  \
0      6537481                          Dolomitenhotel         3.0   
1       174732                                     NaN         NaN   
2       449128                                     NaN         NaN   
3      1177961                                     NaN         NaN   
4       184567                                     NaN         NaN   
5       390933                    Hotel Praded Jesenik         2.0   
6      6629907                                     NaN         NaN   
7      7621602                                     NaN         NaN   
8       887106                                     NaN         NaN   
9      7600445                         Studio Moncoeur         3.0   
10      115079                              Hotel Adam         2.0   
11     5302869                                     NaN         NaN   
12      454523                                     NaN         NaN   
13      332464      

### Part 2 (25 Points)

For this part, we look at the logic and how you solve the problems. 

***Part (a):*** 

    1- You need to find "5" interesting business questions based on the datasets. Please make sure that these quastions are not similar with other groups...     
    2- Write Python code to answer the questions.    
    3- Visualize your results for each question. 
    

***Part (b):***

    Write a 300-word summary of your answers and business insights you get from answering these 5 questions based on your code. Ensure that you have clearly explained why we should care about your questions and your results. Clearly explain your findings.   

***This part will be evaluated based on the following criteria:***

    1. You need to ask five business-relevant questions. (5 points)
    2. You need to answer these five questions using Python and the two datasets. (5 points)
    3. You need to have at least "5" graphs to visualize your insights. (6 points)
    4. Your executive summary should be well-written. (6 points)
    5. Your results and business insights should be interesting and meaningful. (3 points)


**Note**: You may use this cell to write your 5 questions

**Question 1:** 

**Question 2:** 

**Question 3:** 

**Question 4:** 

**Question 5:** 


In [ ]:
# Your code to answer Question 1


In [ ]:
# Your code to answer Question 2


In [ ]:
# Your code to answer Question 3


In [ ]:
# Your code to answer Question 4


In [ ]:
# Your code to answer Question 5


### Executive Summary & Business insights:


**Note**: You need to use the cell below to write your executive summary & business insights.
If you need more space use enter to go to the next line.


## Write here:


### Grading:

PART 1 - 75 points (85 points with the extra credits in the Bonus Question)

- Question 1: 9 points (6 points for part (a) and 3 points for part (b))
- Question 2: 15 points (9 points for part (a) and 6 points for part (b))
- Question 3: 12 points (9 points for part (a) and 3 points for part (b))
- Question 4: 21 points (9 points for part (a) and 12 points for part (b))
- Question 5: 18 points (9 points for part (a) and 9 points for part (b))
- Bonus Question: 10 points (extra credit): (8 points for part (a) and 2 points for part (b))




PART 2 - 25 points 
- You need to ask five business-related questions (5 points).
- You need to answer these five questions using Python and the two datasets (5 points).
- You need to have at least "5" graphs to visualize your insights (6 points).
- Your executive summary should be well-written (6 points).
- Your results and business insights should be interesting and meaningful (3 points).


Good Luck!